## Xarray engine: temporal options

First, we get some forecast GRIB data defined on pressure levels and read it into a GRIB fieldlist.

In [1]:
import earthkit.data as ekd
ds_fl = ekd.from_source("sample", "pl_regular_ll_small.grib")

### Temporal dimensions

The temporal dimensions are defined by the ``time_dim_mode`` option.

#### time_dim_mode=raw

When ``time_dim_mode`` is "raw" the "date", "time" and "step" ecCodes GRIB keys are used to form the temporal dimensions.

In [2]:
ds = ds_fl.to_xarray(time_dim_mode="raw")
ds

<xarray.Dataset> Size: 176kB
Dimensions:    (date: 2, time: 2, step: 2, levelist: 2, latitude: 19,
                longitude: 36)
Coordinates:
  * date       (date) datetime64[ns] 16B 2024-06-03 2024-06-04
  * time       (time) timedelta64[ns] 16B 00:00:00 12:00:00
  * step       (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * levelist   (levelist) int64 16B 500 700
  * latitude   (latitude) float64 152B 90.0 80.0 70.0 60.0 ... -70.0 -80.0 -90.0
  * longitude  (longitude) float64 288B 0.0 10.0 20.0 30.0 ... 330.0 340.0 350.0
Data variables:
    r          (date, time, step, levelist, latitude, longitude) float64 88kB ...
    t          (date, time, step, levelist, latitude, longitude) float64 88kB ...
Attributes:
    param:        t
    class:        od
    stream:       oper
    levtype:      pl
    type:         fc
    expver:       0001
    domain:       g
    number:       0
    Conventions:  CF-1.8
    institution:  ECMWF

#### time_dim_mode=forecast

When ``time_dim_mode`` is "forecast" the "date" and "time" ecCodes GRIB keys are merged to form the dimension "forecats_reference_time". It also adds the "step" dimension based on the "step" key.

In [3]:
# default mode
ds = ds_fl.to_xarray(time_dim_mode="forecast")
ds

<xarray.Dataset> Size: 176kB
Dimensions:                  (forecast_reference_time: 4, step: 2, levelist: 2,
                              latitude: 19, longitude: 36)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 202...
  * step                     (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * levelist                 (levelist) int64 16B 500 700
  * latitude                 (latitude) float64 152B 90.0 80.0 ... -80.0 -90.0
  * longitude                (longitude) float64 288B 0.0 10.0 ... 340.0 350.0
Data variables:
    r                        (forecast_reference_time, step, levelist, latitude, longitude) float64 88kB ...
    t                        (forecast_reference_time, step, levelist, latitude, longitude) float64 88kB ...
Attributes:
    param:        t
    class:        od
    stream:       oper
    levtype:      pl
    type:         fc
    expver:       0001
    domain:       g
    number:       0
    Conventions:  CF-1.8
    institution:  ECMWF

#### time_dim_mode=valid_time

When ``time_dim_mode`` is "valid_time" the only temporal dimension is "valid_time". It is built from the values of the "validityDate" and "validityTime" ecCodes GRIB keys. This dimension can only be generated if each GRIB field has a distinct valid time, so first we need to filter our GRIB data to provide a suitable input for the conversion.

In [4]:
ds = ds_fl.sel(date=20240603, time=0).to_xarray(time_dim_mode="valid_time")
ds

<xarray.Dataset> Size: 44kB
Dimensions:     (valid_time: 2, levelist: 2, latitude: 19, longitude: 36)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 16B 2024-06-03 2024-06-03T06:00:00
  * levelist    (levelist) int64 16B 500 700
  * latitude    (latitude) float64 152B 90.0 80.0 70.0 ... -70.0 -80.0 -90.0
  * longitude   (longitude) float64 288B 0.0 10.0 20.0 ... 330.0 340.0 350.0
Data variables:
    r           (valid_time, levelist, latitude, longitude) float64 22kB ...
    t           (valid_time, levelist, latitude, longitude) float64 22kB ...
Attributes:
    param:        t
    class:        od
    stream:       oper
    levtype:      pl
    type:         fc
    expver:       0001
    date:         20240603
    time:         0
    domain:       g
    number:       0
    Conventions:  CF-1.8
    institution:  ECMWF

### Adding valid_time coord

When ``add_valid_time_dim`` is True it adds coord `valid_time` containing the valid times for all the different temporal dimensions as datetime64. When ``time_dim_mode`` is "valid_time" this coordinate is always added irrespectively of the value of ``add_valid_time_dim``.

In [5]:
ds = ds_fl.to_xarray(time_dim_mode="raw", add_valid_time_coord=True)
ds

<xarray.Dataset> Size: 176kB
Dimensions:     (date: 2, time: 2, step: 2, levelist: 2, latitude: 19,
                 longitude: 36)
Coordinates:
  * date        (date) datetime64[ns] 16B 2024-06-03 2024-06-04
  * time        (time) timedelta64[ns] 16B 00:00:00 12:00:00
  * step        (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * levelist    (levelist) int64 16B 500 700
    valid_time  (date, time, step) datetime64[ns] 64B ...
  * latitude    (latitude) float64 152B 90.0 80.0 70.0 ... -70.0 -80.0 -90.0
  * longitude   (longitude) float64 288B 0.0 10.0 20.0 ... 330.0 340.0 350.0
Data variables:
    r           (date, time, step, levelist, latitude, longitude) float64 88kB ...
    t           (date, time, step, levelist, latitude, longitude) float64 88kB ...
Attributes:
    param:        t
    class:        od
    stream:       oper
    levtype:      pl
    type:         fc
    expver:       0001
    domain:       g
    number:       0
    Conventions:  CF-1.8
    institution:  ECMWF

### Decoding temporal coords

When ``decode_times=True`` (the default) the follwing coordinates will be stored as datetime64:

- coordinates representing the date-like ecCodes keys (e.g. "date", "validityDate" etc.)
- datetime coordinates (e.g. "forecast_reference_time" etc.)

When ``decode_timedelta=True`` (the default) the following coordinates will be stored as timedelta64:

- coordinates representing the time-like ecCodes keys (e.g. "time", "validityTime" etc.)
- duration-like coordinates (e.g. "step")

In [6]:
ds = ds_fl.to_xarray(time_dim_mode="raw", decode_times=True, decode_timedelta=True)
ds.coords

Coordinates:
  * date       (date) datetime64[ns] 16B 2024-06-03 2024-06-04
  * time       (time) timedelta64[ns] 16B 00:00:00 12:00:00
  * step       (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * levelist   (levelist) int64 16B 500 700
  * latitude   (latitude) float64 152B 90.0 80.0 70.0 60.0 ... -70.0 -80.0 -90.0
  * longitude  (longitude) float64 288B 0.0 10.0 20.0 30.0 ... 330.0 340.0 350.0

When ``decode_times=False`` the following rules apply:

- coordinates representing date-like ecCodes keys (e.g. "date", "validityDate" etc.) will store the native GRIB int values (as yyyymmdd)
- datetime coordinates (e.g. "forecast_reference_time" etc.) will store datetime64 values

When ``decode_timedelta=False`` the following rules apply:

- coordinates representing the time-like ecCodes keys (e.g. "time", "validityTime" etc.) will store the native GRIB int values (as 100*hours + minutes)
- duration-like (e.g. "step") coordinates will store int values with units indicated by the coordinate attribute "units"

In [7]:
ds = ds_fl.to_xarray(time_dim_mode="raw", decode_times=False, decode_timedelta=False)
ds.coords

Coordinates:
  * date       (date) int64 16B 20240603 20240604
  * time       (time) int64 16B 0 1200
  * step       (step) int64 16B 0 6
  * levelist   (levelist) int64 16B 500 700
  * latitude   (latitude) float64 152B 90.0 80.0 70.0 60.0 ... -70.0 -80.0 -90.0
  * longitude  (longitude) float64 288B 0.0 10.0 20.0 30.0 ... 330.0 340.0 350.0

In [8]:
ds.coords["step"].attrs

{'standard_name': 'forecast_period',
 'long_name': 'time since forecast_reference_time',
 'units': 'hours'}